### Text Representation and Sentiment Analysis
##### Jan 15th 2023
##### William Lu


In [1]:
import pandas as pd
import numpy as np
import nltk
#nltk.download()
import re
from bs4 import BeautifulSoup
 

In [2]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [3]:
# Download the zip file and then add to the path

In [4]:
path  = '/Users/William/Downloads/amazon_reviews_us_Beauty_v1_00.tsv'
df = pd.read_csv(path,on_bad_lines='skip',sep='\t')
df

/var/folders/wf/cgnnphfs6fjbzlg451nf6y8w0000gn/T/ipykernel_30786/2583716964.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path,on_bad_lines='skip',sep='\t')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094302,US,50113639,RZ7RZ02MTP4SL,B000050B70,185454094,Conair NE150NSCS Cordless Nose and Ear Hair Tr...,Beauty,5,10.0,10.0,N,N,Great Little Grooming Tool,After watching my Dad struggle with his scisso...,2000-11-12
5094303,US,52940456,R2IRC0IZ8YCE5T,B000050FF2,678848064,Homedics Envirascape Sound Spa Alarm Clock Radio,Beauty,3,23.0,23.0,N,N,Not bad for the price,"Like most sound machines, the sounds choices a...",2000-11-07
5094304,US,47587881,R1U4ZSXOD228CZ,B000050B6U,862195513,Conair Instant Heat Curling Iron,Beauty,5,89.0,97.0,N,N,Best Curling Iron Ever,I bought this product because it indicated 30 ...,2000-11-02
5094305,US,53047750,R3SFJLZE09URWM,B000050FDE,195242894,Oral-B Professional Care 1000 Power Toothbrush,Beauty,5,10.0,10.0,N,N,"The best electric toothbrush ever, REALLY!",We have used Oral-B products for 15 years; thi...,2000-11-01


## Keep Reviews and Ratings

In [5]:
# Our goal is to train sentiment analysis classifiers that can predict the rating value for a given review.
df2 = df.copy()
dff = df2[['review_body','star_rating']]
dff

,review_body,star_rating
0,"Love this, excellent sun block!!",5
1,The great thing about this cream is that it do...,5
2,"Great Product, I'm 65 years old and this is al...",5
3,I use them as shower caps & conditioning caps....,5
4,This is my go-to daily sunblock. It leaves no ...,5
...,...,...
5094302,After watching my Dad struggle with his scisso...,5
5094303,"Like most sound machines, the sounds choices a...",3
5094304,I bought this product because it indicated 30 ...,5
5094305,We have used Oral-B products for 15 years; thi...,5


 ## We form three classes and select 20000 reviews randomly from each class.



In [6]:
#dff['star_rating'] = dff['star_rating'].astype(int)
class_one_score1 = dff.loc[dff['star_rating']=='1']
class_one_score2 = dff.loc[dff['star_rating']=='2']
class_one_label = [class_one_score1,class_one_score2]
class_one =pd.concat(class_one_label,ignore_index=True)
class_one['class_label'] = '1'
class_two_label = dff.loc[dff['star_rating']=='3']
class_two = class_two_label.copy()
class_two['class_label'] = '2'
class_three_score1 = dff.loc[dff['star_rating']=='4']
class_three_score2 = dff.loc[dff['star_rating']=='5']
class_three_label = [class_three_score1,class_three_score2]
class_three =pd.concat(class_three_label,ignore_index=True)
class_three['class_label'] = '3'

In [7]:
# randonly select 20000 rows for each class_label
one = class_one.sample(n = 20000)
two = class_two.sample(n = 20000)
three = class_three.sample(n = 20000)

In [8]:
# combine the three dataframes together and form a balanced dataset
df_bal_class = [one,two,three]
df_bal = pd.concat(df_bal_class,ignore_index = True)
df_bal

,review_body,star_rating,class_label
0,The brush hair some hard,2,1
1,We had tried several things on our son so he w...,2,1
2,Did not help me... In just two weeks my scalp ...,1,1
3,This particular e.l.f. Beauty Book was an Amaz...,1,1
4,Allergies qil came out all over your bodies on...,1,1
...,...,...,...
59995,l love it! Wish l'd bought it yrs ago. Does ev...,5,3
59996,Love this stuff. Quick delivery.,5,3
59997,Peoduct came on time in good condition<br />Ga...,4,3
59998,good,5,3


# Data Cleaning



In [9]:
df_bal['review_body'] = df_bal['review_body'].astype(str)
pre_length = df_bal['review_body'].apply(len).mean()
#print(f'Before data cleaning, the average lengh of reviws in terms of characters is: {pre_length:.3f}')

In [10]:
import contractions
import math

In [11]:
df_bal['review_body'] = df_bal['review_body'].astype(str)
# convert all reviews to lowercase
df_bal['review_body']= df_bal['review_body'].str.lower()
# remove urls from the reviews
df_bal["review_body"] = df_bal["review_body"].str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip()
# remove htmls tag from the reviews
df_bal['review_body'] = df_bal['review_body'].str.replace(r'<[^<>]*>', '', regex=True)
# remove numerics
df_bal['review_body'] = df_bal['review_body'].str.replace('\d+', '',regex=True)
# remove non-alphabetical characters
df_bal['review_body'] = df_bal['review_body'].str.replace('[^a-zA-Z0-9]', ' ', regex=True)
# perform contractions
df_bal['review_body'] = df_bal['review_body'].apply(lambda x: contractions.fix(x))
# remove extra space
df_bal['review_body'] = df_bal['review_body'].str.strip()
df_bal

/var/folders/wf/cgnnphfs6fjbzlg451nf6y8w0000gn/T/ipykernel_30786/2004679854.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_bal["review_body"] = df_bal["review_body"].str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip()


,review_body,star_rating,class_label
0,the brush hair some hard,2,1
1,we had tried several things on our son so he w...,2,1
2,did not help me in just two weeks my scalp ...,1,1
3,this particular e l f beauty book was an amaz...,1,1
4,allergies qil came out all over your bodies on...,1,1
...,...,...,...
59995,l love it wish l d bought it yrs ago does ev...,5,3
59996,love this stuff quick delivery,5,3
59997,peoduct came on time in good conditiongave a...,4,3
59998,good,5,3


In [12]:
post_length = df_bal['review_body'].apply(len).mean()
print(f'Before and After data cleaning, the average length of reviews in terms of characters is:{pre_length},{post_length}')

Before and After data cleaning, the average length of reviews in terms of characters is:189.23125,186.38478333333333


# Pre-processing

## remove the stop words 

In [13]:
pre_length_pr = df_bal['review_body'].apply(len).mean()
#print(f'After data cleaning, the average lengh of reviws in terms of characters is: {post_length_pr:.3f}')

In [14]:
# remove the stop words
# from nltk.corpus import stopwords
stop = nltk.corpus.stopwords.words('english')
df_bal['review_body withno stopwords'] = df_bal['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_bal

,review_body,star_rating,class_label,review_body withno stopwords
0,the brush hair some hard,2,1,brush hair hard
1,we had tried several things on our son so he w...,2,1,tried several things son would stop sucking th...
2,did not help me in just two weeks my scalp ...,1,1,help two weeks scalp became inflamed thought p...
3,this particular e l f beauty book was an amaz...,1,1,particular e l f beauty book amazon prime day ...
4,allergies qil came out all over your bodies on...,1,1,allergies qil came bodies used
...,...,...,...,...
59995,l love it wish l d bought it yrs ago does ev...,5,3,l love wish l bought yrs ago everything promis...
59996,love this stuff quick delivery,5,3,love stuff quick delivery
59997,peoduct came on time in good conditiongave a...,4,3,peoduct came time good conditiongave felt litt...
59998,good,5,3,good


## perform lemmatization  

In [15]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
# reference: https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing
wordnet_map = {"N":wordnet.NOUN,"V":wordnet.VERB,"J":wordnet.ADJ,"R":wordnet.ADV}
# define a lemmatization function and then we can pass in a panda series to perform lemmatization          
def lemmatization(series):
    pos_tagged = nltk.pos_tag(series.split())
    return ' '.join([lmtzr.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged])
df_bal['review_body final'] = df_bal['review_body withno stopwords'].apply(lambda x: lemmatization(x)) #document
df_bal

,review_body,star_rating,class_label,review_body withno stopwords,review_body final
0,the brush hair some hard,2,1,brush hair hard,brush hair hard
1,we had tried several things on our son so he w...,2,1,tried several things son would stop sucking th...,tried several thing son would stop suck thumb ...
2,did not help me in just two weeks my scalp ...,1,1,help two weeks scalp became inflamed thought p...,help two week scalp become inflamed thought pa...
3,this particular e l f beauty book was an amaz...,1,1,particular e l f beauty book amazon prime day ...,particular e l f beauty book amazon prime day ...
4,allergies qil came out all over your bodies on...,1,1,allergies qil came bodies used,allergy qil come body use
...,...,...,...,...,...
59995,l love it wish l d bought it yrs ago does ev...,5,3,l love wish l bought yrs ago everything promis...,l love wish l buy yrs ago everything promise l...
59996,love this stuff quick delivery,5,3,love stuff quick delivery,love stuff quick delivery
59997,peoduct came on time in good conditiongave a...,4,3,peoduct came time good conditiongave felt litt...,peoduct come time good conditiongave felt litt...
59998,good,5,3,good,good


In [16]:
post_length_pr = df_bal['review_body final'].apply(len).mean()
print(f'Before and After data processing, the average length of reviews in terms of characters is:{pre_length_pr},{post_length_pr}')

Before and After data processing, the average length of reviews in terms of characters is:186.38478333333333,105.8852


# TF-IDF Feature Extraction

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
# turn the dataframe column to a list called corpus
#corpus = df_bal['review_body str'].tolist()
Vectorizer = TfidfVectorizer(min_df = 15,max_df=0.9)
tfidf_X = Vectorizer.fit_transform(df_bal['review_body final'])
tfidf_features = Vectorizer.get_feature_names_out()

In [18]:
tfidf_features

array(['aa', 'ability', 'able', ..., 'zipper', 'zit', 'zone'],
      dtype=object)

In [19]:
tfidf_features.shape

(3745,)

In [20]:
df_bal['review_body final'][6]

'use almost month see improvement line wrinkle hydration stop skin start get red irritated sensitive skin vitamin c ascorbic acid maybe know'

In [21]:
tfidf_X.todense().shape

(60000, 3745)

# Perceptron

In [22]:
# train test split the data
from sklearn.model_selection import train_test_split
X1 = tfidf_X # tfidf feature vector
y = df_bal['class_label']
X_train, X_test, y_train, y_test = train_test_split(X1,y ,
                                   random_state=104, 
                                   test_size=0.2, 
                                   shuffle=True)

In [23]:
X_train

<48000x3745 sparse matrix of type '<class 'numpy.float64'>'
	with 662414 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn import metrics

In [25]:
from sklearn.linear_model import Perceptron

In [26]:
pctr = Perceptron(random_state = 102) # Shuffle is set to True by default
pctr.fit(X_train,y_train)

Perceptron(random_state=102)

In [27]:
y_predicted_pctr = pctr.predict(X_test)
accuracy_pctr = accuracy_score(y_test,y_predicted_pctr)
f1_pctr = f1_score(y_test,y_predicted_pctr,average='macro')
recall_pctr = recall_score(y_test,y_predicted_pctr,average='macro')
precision_pctr = precision_score(y_test,y_predicted_pctr,average='macro')
#print('Accuracy is :',accuracy_pctr)
#print('Average f1 score is :',f1_pctr)
#print('Average recall score is :',recall_pctr)
#print('Average precision score is :',precision_pctr)

In [28]:
print(metrics.classification_report(y_test, y_predicted_pctr))

              precision    recall  f1-score   support

           1       0.62      0.60      0.61      3995
           2       0.52      0.56      0.54      4087
           3       0.71      0.67      0.69      3918

    accuracy                           0.61     12000
   macro avg       0.62      0.61      0.61     12000
weighted avg       0.62      0.61      0.61     12000



# SVM

In [30]:
#from sklearn import svm
from sklearn.svm import LinearSVC
SVM = LinearSVC()
SVM.fit(X_train,y_train)

LinearSVC()

In [31]:
y_predicted_svm = SVM.predict(X_test)
accuracy_svm = accuracy_score(y_test,y_predicted_svm)
f1_svm = f1_score(y_test,y_predicted_svm,average='macro')
recall_svm = recall_score(y_test,y_predicted_svm,average='macro')
precision_svm = precision_score(y_test,y_predicted_svm,average='macro')
#print('Accuracy is :',accuracy_svm)
#print('Average f1 score is :',f1_svm)
#print('Average recall score is :',recall_svm)
#print('Average precision score is :',precision_svm)

In [32]:
print(metrics.classification_report(y_test, y_predicted_svm))

              precision    recall  f1-score   support

           1       0.66      0.70      0.68      3995
           2       0.60      0.54      0.57      4087
           3       0.73      0.77      0.75      3918

    accuracy                           0.67     12000
   macro avg       0.66      0.67      0.67     12000
weighted avg       0.66      0.67      0.66     12000



# Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(multi_class='multinomial',solver='lbfgs', max_iter =1000)
lr.fit(X_train,y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [34]:
y_predicted_lr = lr.predict(X_test)
accuracy_lr = accuracy_score(y_test,y_predicted_lr)
f1_lr = f1_score(y_test,y_predicted_lr,average='macro')
recall_lr = recall_score(y_test,y_predicted_lr,average='macro')
precision_lr = precision_score(y_test,y_predicted_lr,average='macro')
#print('Accuracy is :',accuracy_lr)
#print('Average f1 score is :',f1_lr)
#print('Average recall score is :',recall_lr)
#print('Average precision score is :',precision_lr)

In [35]:
print(metrics.classification_report(y_test, y_predicted_lr))

              precision    recall  f1-score   support

           1       0.67      0.71      0.69      3995
           2       0.61      0.57      0.59      4087
           3       0.76      0.77      0.76      3918

    accuracy                           0.68     12000
   macro avg       0.68      0.68      0.68     12000
weighted avg       0.68      0.68      0.68     12000



# Naive Bayes

In [36]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train,y_train)
y_predicted_mnb = mnb.predict(X_test)
accuracy_mnb = accuracy_score(y_test,y_predicted_mnb)
f1_mnb = f1_score(y_test,y_predicted_mnb,average='macro')
recall_mnb = recall_score(y_test,y_predicted_mnb,average='macro')
precision_mnb = precision_score(y_test,y_predicted_mnb,average='macro')
#print('Accuracy is :',accuracy_mnb)
#print('Average f1 score is :',f1_mnb)
#print('Average recall score is :',recall_mnb)
#print('Average precision score is :',precision_mnb)

In [37]:
print(metrics.classification_report(y_test, y_predicted_mnb))

              precision    recall  f1-score   support

           1       0.69      0.66      0.67      3995
           2       0.59      0.59      0.59      4087
           3       0.73      0.75      0.74      3918

    accuracy                           0.67     12000
   macro avg       0.67      0.67      0.67     12000
weighted avg       0.67      0.67      0.67     12000

